In [18]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()
client = OpenAI()

print("API Loaded?", os.getenv("OPENAI_API_KEY") is not None)

API Loaded? True


In [19]:
df = pd.read_csv("Dataset/student_performance_with_clusters.csv")
df.head()

,Student ID,Study Hours per Week,Attendance Rate,Previous Grades,Participation in Extracurricular Activities,Passed,Cluster
0,S00002,0.450000,0.666261,0.272822,0,No,0
1,S00004,0.647619,0.553799,0.282212,1,No,1
2,S00005,0.438095,0.629787,0.335942,0,No,0
3,S00006,0.438095,0.535562,0.318206,1,Yes,1
4,S00007,0.654762,0.321581,0.444966,0,Yes,0


In [20]:
resp = client.responses.create(
    model="gpt-4o-mini",
    input="Say: API is working!"
)
print(resp.output_text)

API is working!


In [21]:
def generate_advice_template1(row):
    prompt = f"""
    You are an academic advisor. Provide a short, simple performance advice 
    for a student based on the following details:

    Study Hours: {row['Study Hours per Week']}
    Attendance Rate: {row['Attendance Rate']}
    Previous Grades: {row['Previous Grades']}
    Participation in Activities: {row['Participation in Extracurricular Activities']}
    Cluster Group: {row['Cluster']}

    Keep the advice in 2 sentences maximum.
    """

    resp = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )
    return resp.output_text

In [22]:
def generate_advice_template2(row):
    study = row["Study Hours per Week"]
    attendance = row["Attendance Rate"]
    grades = row["Previous Grades"]
    cluster = row["Cluster"]
    
    return f"""
### Performance Summary
The student shows performance across key indicators with study hours of {study:.2f}, attendance rate of {attendance:.2f}, and grades averaging {grades:.2f}. This reflects their overall engagement and learning progress.

### Biggest Weakness
The primary weakness appears to be related to **academic performance**, specifically the grade average of {grades:.2f}. This suggests challenges in understanding or keeping up with the material.

### Strength
The student demonstrates a relative strength in **attendance** (rate: {attendance:.2f}). Consistent attendance can support improved understanding and academic success.

### Recommended Action Plan
- Increase daily or weekly study time to reinforce understanding.
- Engage more in class activities and discussions.
- Seek support resources such as tutoring or study groups.
- Join at least one extracurricular activity to develop broader skills.
- Set measurable academic goals and track progress regularly.

### How their Cluster ({cluster}) Typically Performs
Students in Cluster {cluster} generally have performance patterns similar to this student. This cluster often shows lower engagement and needs targeted interventions to improve study habits, motivation, and academic outcomes.
"""

In [23]:
sample = df.head(3).copy()

sample["Advice_Template1"] = sample.apply(generate_advice_template1, axis=1)
sample["Advice_Template2"] = sample.apply(generate_advice_template2, axis=1)

sample

,Student ID,Study Hours per Week,Attendance Rate,Previous Grades,Participation in Extracurricular Activities,Passed,Cluster,Advice_Template1,Advice_Template2
0,S00002,0.450000,0.666261,0.272822,0,No,0,"To improve your academic performance, increase...",\n### Performance Summary\nThe student shows p...
1,S00004,0.647619,0.553799,0.282212,1,No,1,"To improve your academic performance, increase...",\n### Performance Summary\nThe student shows p...
2,S00005,0.438095,0.629787,0.335942,0,No,0,"To improve your academic performance, focus on...",\n### Performance Summary\nThe student shows p...


In [24]:
for i, row in sample.iterrows():
    print(f"\n=== Student {row['Student ID']} ===")
    print("\nTemplate 1 Output:")
    print(row["Advice_Template1"])
    print("\nTemplate 2 Output:")
    print(row["Advice_Template2"])
    print("\n------------------------------")


=== Student S00002 ===

Template 1 Output:
To improve your academic performance, increase your study hours and attendance to create a more consistent routine. Additionally, consider joining activities to enhance your engagement and learning experience.

Template 2 Output:

### Performance Summary
The student shows performance across key indicators with study hours of 0.45, attendance rate of 0.67, and grades averaging 0.27. This reflects their overall engagement and learning progress.

### Biggest Weakness
The primary weakness appears to be related to **academic performance**, specifically the grade average of 0.27. This suggests challenges in understanding or keeping up with the material.

### Strength
The student demonstrates a relative strength in **attendance** (rate: 0.67). Consistent attendance can support improved understanding and academic success.

### Recommended Action Plan
- Increase daily or weekly study time to reinforce understanding.
- Engage more in class activities a

(1) Template Comparison & Analysis

When analyzing the results produced by the two prompt templates, distinct differences are apparent in terms of quality, detail, and relevance.

Template 1 generates brief, straightforward guidance, usually restricted to one or two sentences, concentrating solely on the key actions for enhancement like boosting study time or improving attendance. This allows for rapid and efficient responses in scenarios needing prompt feedback, yet it falls short in depth and contextual comprehension.

In comparison, Template 2 provides a significantly more detailed and organized reply. It offers a detailed performance summary, pinpointing weaknesses (like poor grades or inadequate study hours), emphasizing strengths (such as good attendance), and presenting a multi-step plan for actionable improvement. Furthermore, it offers a cluster-based analysis, providing the student with a better understanding of how their performance measures up within their subgroup. This degree of detail renders Template 2 far more informative, more contextually appropriate, and more effective for substantial academic advising.

(2) Final Template Selection & Justification

The chosen prompt template for incorporation into the system is Template 2. This option is chosen for its capacity to provide a deeper, more organized, and educationally significant analysis in comparison to Template 1. Template 1 delivers swift and straightforward feedback, whereas Template 2 gives a more thorough insight into student performance by highlighting strengths, weaknesses, and specific steps for improvement. Its organized design showcasing a performance overview, in-depth rationale, and a cluster comparison improves clarity and facilitates knowledgeable academic choices. Hence, Template 2 aligns more closely with the system's objective of providing thorough, top notch guidance that significantly aids student advancement.